# ATP Tennis Data - Feature Engineering

In previous notebook, we pre-processed the ATP match data by:
* dropping columns with little data
* cleaning string columns
* imputing any data that we can reasonably impute

For this notebook, we will use the data saved from pre-processing and start doing some basic feature engineering so we can start feeding our data to our models to see how we do. My goal is to start training models and see how they do as soon as possible, so we will We will do the following:
* one hot encode any categorical data - ie, tournament surface, round, winner_ioc, loser_ioc
* extra month from the tournament date
* label each row with 0 as losing and 1 as winning
* remove any remaining columns that we

In a future notebook, I plan on implementing more advanced feature engineering with:
* look up matchup history for players
* match-up stats for pervious matchups
* add match player record leading up to the match for each player
* look up player stats leading up to match for each player

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set()